# In this notebook we will covere:

1. Introduction to Reasoning Models and GRPO
2. Installing Dependencies
3. Loading and Testing Models
4. Fine-Tuning with GRPO
5. Experimenting with Different Models (DeepSeek, GPT, Claude)
6. Final Model Comparison
7. Analyzing Results



# 1: Introduction to Reasoning Models and GRPO
## What is GRPO?
### Group Relative Policy Optimization (GRPO) is an advanced reinforcement learning algorithm that enables models to:

1. Autonomously improve reasoning by generating and scoring responses.
2. Use group averages as benchmarks for reinforcement.

### Reasoning models are designed to process queries, derive logical conclusions, and provide accurate responses. Hugging Face's Transformers library allows us to:

1. Fine-tune models for reasoning tasks.
2. Experiment with various pre-trained models like GPT, T5, and BERT.

#  2: Installing Dependencies


In [5]:
!pip install -q transformers datasets accelerate


#  3: Loading and Testing Models

## e use Hugging Face's AutoModelForCausalLM for testing GPT-based models, Lamma, Qwin, DeepSeek..

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test the model
query = "What is the capital of France?"
input_ids = tokenizer.encode(query, return_tensors="pt")
output = model.generate(input_ids, max_length=50)
print(f"Query: {query}\nResponse: {tokenizer.decode(output[0], skip_special_tokens=True)}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Query: What is the capital of France?
Response: What is the capital of France?

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is


# 4: Fine-Tuning a Model with Hugging Face
## We use Hugging Face's dataset (squad [link text](https://huggingface.co/datasets/rajpurkar/squad)) library to load a dataset for fine-tuning.
### Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

In [7]:
from datasets import load_dataset

# Load a dataset for fine-tuning
dataset = load_dataset("squad")

# Show a sample from the dataset
print(dataset["train"][0])


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [ ]:
from transformers import default_data_collator

# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD

# Preprocessing function
def preprocess_function(examples):
    # Combine context and question
    inputs = [c + " " + q for c, q in zip(examples["context"], examples["question"])]

    # Extract answers (handle missing cases)
    targets = [a["text"][0] if a["text"] else "" for a in examples["answers"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets (answers)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    # Replace `[PAD]` tokens in labels with `-100` to ignore in loss function
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    # Ensure labels match input length (truncate if needed)
    for i in range(len(labels["input_ids"])):
        if len(labels["input_ids"][i]) > len(model_inputs["input_ids"][i]):
            labels["input_ids"][i] = labels["input_ids"][i][:len(model_inputs["input_ids"][i])]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Show sample
print("Input IDs:", tokenized_datasets["train"][0]["input_ids"])
print("Labels:", tokenized_datasets["train"][0]["labels"])


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

## Fine-tune the GPT-2 model on a QA dataset.

In [26]:
# Check sample tokenized input-label pairs
print("Input IDs:", tokenized_datasets["train"][0]["input_ids"])
print("Labels:", tokenized_datasets["train"][0]["labels"])


Input IDs: [19895, 5712, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774, 11819, 338, 3869, 29500, 318, 257, 10861, 15207, 286, 262, 5283, 5335, 13, 34528, 287, 2166, 286, 262, 8774, 11819, 290, 6476, 340, 11, 318, 257, 15317, 15207, 286, 1951, 351, 5101, 510, 49309, 351, 262, 8177, 366, 37522, 578, 1215, 2185, 16543, 2516, 1911, 7406, 284, 262, 8774, 11819, 318, 262, 32520, 3970, 286, 262, 17380, 8894, 13, 34528, 2157, 262, 37792, 3970, 318, 262, 10299, 33955, 11, 257, 37919, 1295, 286, 11443, 290, 14580, 13, 632, 318, 257, 30069, 286, 262, 7128, 33955, 379, 406, 454, 8906, 11, 4881, 810, 262, 5283, 5335, 1128, 7241, 306, 4120, 284, 9281, 6206, 324, 5857, 311, 12944, 343, 516, 287, 1248, 3365, 13, 1629, 262, 886, 286, 262, 1388, 3708, 357, 392, 287, 257, 1277, 1627, 326, 20417, 832, 513, 25827, 290, 262, 3561, 31390, 828, 318, 257, 2829, 11, 3660, 7815, 15207, 286, 5335, 13, 1675, 4150, 750, 262, 5283, 5335, 7910, 1656, 287, 1248, 3365, 287, 406, 454, 8906, 4881, 

In [23]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    eval_strategy="epoch",  # Updated to avoid deprecation warnings
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    num_train_epochs=15,
    weight_decay=0.01,
    remove_unused_columns=False,  # Prevents automatic column dropping
    report_to="none",  # Disable Weights & Biases if not needed
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(100)),  # Use subset for fast training
    eval_dataset=tokenized_datasets["validation"].select(range(50)),
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# Train the model
trainer.train()


<ipython-input-23-19e68bb0adfb>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,9.545432


TrainOutput(global_step=25, training_loss=9.812628173828125, metrics={'train_runtime': 33.3953, 'train_samples_per_second': 2.994, 'train_steps_per_second': 0.749, 'total_flos': 26129203200000.0, 'train_loss': 9.812628173828125, 'epoch': 1.0})